In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked "
            "to extract, return null for the attribute's value.",
        ),
        MessagesPlaceholder("examples"),
        ("human","{text}"),
    ]
)

In [2]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


class Person(BaseModel):
    """Information about a person."""
    name: Optional[str] = Field(..., description="The name of the person")
    hair_color: Optional[str] = Field(
        ..., description="The color of the peron's eyes if known"
    )
    height_in_meters: Optional[str] = Field(..., description="Height in METERs")


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [3]:
import uuid
from typing import Dict, List, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.pydantic_v1 import BaseModel, Field

In [4]:
class Example(TypedDict):
    """A representation of an example consisting of text input and expected tool calls.

    For extraction, the tool calls are represented as instances of pydantic model.
    """
    input:str
    tool_calls: List[BaseModel]

In [5]:
def tool_example_to_messages(example:Example) -> List[BaseMessage]:
    """Convert an example into a list of messages that can be fed into an LLM.

    This code is an adapter that converts our example to a list of messages
    that can be fed into a chat model.

    The list of messages per example corresponds to:

    1) HumanMessage: contains the content from which content should be extracted.
    2) AIMessage: contains the extracted information from the model
    3) ToolMessage: contains confirmation to the model that the model requested a tool correctly.

    The ToolMessage is required because some of the chat models are hyper-optimized for agents
    rather than for an extraction use case.
    """
    messages: List[BaseMessage] = [HumanMessage(content=example['input'])]
    openai_tool_calls = []
    for tool_call in example['tool_calls']:
        openai_tool_calls.append(
            {
                "id":str(uuid.uuid4()),
                "type":"function",
                "function":{
                    "name":tool_call.__class__.__name__,
                    "arguments":tool_call.json(),
                }
            }
        )
        messages.append(
            AIMessage(content="",additional_kwargs={"tool_calls":openai_tool_calls})
        )
        tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
        ] * len(openai_tool_calls)
        for output,tool_call in zip(tool_outputs, openai_tool_calls):
            messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
        return messages
    

In [6]:
examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep. There are many fish in it.",
        Person(name=None, height_in_meters=None, hair_color=None, country=None),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Person(name="Fiona", height_in_meters=None, hair_color=None,country=["France","Spain"]),
    ),
]


In [8]:
messages = []
for text,tool_call in examples:
    # print("text-->",text)
    # print("tool_call -->",tool_call)
    messages.extend(
        tool_example_to_messages({"input": text, "tool_calls": [tool_call]})
    )

In [9]:
prompt.invoke({"text": "this is some text", "examples": messages})

ChatPromptValue(messages=[SystemMessage(content="You are an expert extraction algorithm. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value."), HumanMessage(content="The ocean is vast and blue. It's more than 20,000 feet deep. There are many fish in it."), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'a97a80af-e525-4bda-8768-562a98d6f743', 'type': 'function', 'function': {'name': 'Person', 'arguments': '{"name": null, "hair_color": null, "height_in_meters": null}'}}]}), ToolMessage(content='You have correctly called this tool.', tool_call_id='a97a80af-e525-4bda-8768-562a98d6f743'), HumanMessage(content='Fiona traveled far from France to Spain.'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '45221bca-a84b-480b-8e12-7b6148bb94de', 'type': 'function', 'function': {'name': 'Person', 'arguments': '{"name": "Fiona", "hair_color": null, "height_in_meters": nu

In [12]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [14]:
runnable = prompt | llm.with_structured_output(
    schema=Data,
    method="function_calling",
    include_raw=False
)

In [15]:
#without examples 

for _ in range(5):
    text = "The solar system is large, but earth has only 1 moon."
    print(runnable.invoke({"text": text, "examples": []}))

people=[Person(name='Earth', hair_color='blue', height_in_meters='12742'), Person(name='Moon', hair_color='gray', height_in_meters='3475')]
people=[Person(name='Earth', hair_color='Blue', height_in_meters='1.0')]
people=[Person(name='Earth', hair_color='Blue', height_in_meters='6371')]
people=[Person(name='Earth', hair_color='blue', height_in_meters='1.5')]
people=[Person(name='Earth', hair_color='Blue', height_in_meters='6371')]


In [16]:
#with exampes
for _ in range(5):
    text = "The solar system is large, but earth has only 1 moon."
    print(runnable.invoke({"text": text, "examples": messages}))

people=[Person(name='earth', hair_color=None, height_in_meters=None)]
people=[Person(name='Earth', hair_color=None, height_in_meters=None)]
people=[Person(name='earth', hair_color=None, height_in_meters=None)]
people=[Person(name='Fiona', hair_color='blonde', height_in_meters='1.75')]
people=[Person(name='Alice', hair_color='brown', height_in_meters='1.65'), Person(name='Bob', hair_color='black', height_in_meters='1.80'), Person(name='Charlie', hair_color='blonde', height_in_meters='1.70')]


In [17]:
runnable.invoke(
    {
        "text": "My name is Harrison. My hair is black.",
        "examples": messages,
    }
)

Data(people=[Person(name='Harrison', hair_color='black', height_in_meters='null')])

In [21]:
####123456
